In [44]:
#### Now since this is a new file we need to import other modules (YAY!) including our R' emulator
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

### and let's not forget the usual gang:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
import os 

In [45]:
#### Before we get started in an R' environment. We need to talk about something important.
#### Rentrez does not use SNP ID to search. It uses UID. As in, they don't use 'rs2000' 
#### but rather '2000'. So we need to use our previous output and split the function.
Output_GeneSNP_file = pd.read_csv('Resources/OutputFromExtractionCode/Editable_Gene_And_dbSNP_ID_Merged.csv')
Output_GeneSNP_file.head(10)

output_path = 'Resources/OutputFromExtractionCode'

In [46]:
### Now we split it. 
Split_dbSNP_ID = Output_GeneSNP_file
Split_dbSNP_ID[['rs', 'UID']] = Output_GeneSNP_file['SNP ID'].str.split('rs', expand = True)
Split_dbSNP_ID.head(10)

#And delete the rs column:
UID_SNP = Split_dbSNP_ID.drop(columns=['rs'])
UID_SNP.head(10)

## Now we have the UID

,GENE,SNP ID,UID
0,CDKN2B-AS1,rs4977574,4977574
1,LPA,rs10455872,10455872
2,ZNF335,rs3827066,3827066
3,CELSR2,rs12740374,12740374
4,LINC00540 - FTH1P7,rs7994761,7994761
5,APOE,rs429358,429358
6,SMARCA4,rs73015011,73015011
7,ADAMTS8,rs4936098,4936098
8,CHRNA5,rs17486278,17486278
9,CDKN2B-AS1,rs2891168,2891168


In [47]:
### Let's import RENTREZ to our python environment
rentrez = importr('rentrez')

### You'll need to provide an email address to use Entrez tools. BTW Hot tip: 
### If you have multiple lines, using triple quotes is cleaner and easier to read:

robjects.r('''
library(rentrez)
entrez_email <- "kenji.ponsm@gmail.com"
''')

In [48]:
### What can we look for with entrez
snp_database = robjects.r('entrez_db_searchable("snp")')
print(snp_database)

Searchable fields for database 'snp'
  ALL 	 All terms from all searchable fields 
  UID 	 Unique number assigned to publication 
  FILT 	 Limits the records 
  RS 	 Clustered SNP ID (rs) 
  CHR 	 chromosomes 
  GENE 	 locus link symbol 
  HAN 	 Submitter Handle 
  ACCN 	 nucleotide accessions 
  GENE_ID 	 Gene ID 
  FXN 	 dbSNP Functional consequence class 
  GTYP 	 Genotype info 
  SS 	 Submitter ID 
  VARI 	 Allele 
  SCLS 	 SNP class 
  CPOS 	 Chromosome base position 
  WORD 	 Free text associated with record 
  SIDX 	 SNP Index 
  CLIN 	 Variations with clinical effects or significances 
  GMAF 	 Minor Allele Frequency derived from global population (ie. 1000G) 
  VALI 	 Validation status 
  CPOS_GRCH37 	 Chromosome base position on previous assembly version 
  ORGN 	 Organism 
  ALFA_EUR 	 ALFA European Minor Allele Frequency 
  ALFA_AFR 	 ALFA African population Minor Allele Frequency 
  ALFA_ASN 	 ALFA Asian population Minor Allele Frequency 
  ALFA_LAC 	 ALFA Latin American 1

In [49]:
## Let's give it a test shall we? Let's say I want the information of rs429358 (GENE: APOE)
print(UID_SNP.iloc[0,2])
test_snp = UID_SNP.iloc[0,2]
search_test = rentrez.entrez_summary(db='snp', id=test_snp)
search_test_dict = {key: str(search_test.rx2(key) [0]) for key in search_test.names}

search_test_df = pd.DataFrame([search_test_dict])

search_test_df.head(10)



4977574


,uid,snp_id,allele_origin,global_mafs,global_population,global_samplesize,suspected,clinical_significance,genes,acc,...,allele,snp_class,chrpos,chrpos_prev_assm,text,snp_id_sort,clinical_sort,cited_sort,chrpos_sort,merged_sort
0,4977574,4977574,,"[1] ""1000Genomes"" ""ALSPAC"" ""Chilea...",,,,risk-factor,"[1] ""CDKN2B-AS1""\n",NC_000009.12,...,D,snv,9:22098575,9:22098574,,0004977574,1,,0022098575,0


In [50]:
print(search_test.names)


 [1] "uid"                   "snp_id"                "allele_origin"        
 [4] "global_mafs"           "global_population"     "global_samplesize"    
 [7] "suspected"             "clinical_significance" "genes"                
[10] "acc"                   "chr"                   "handle"               
[13] "spdi"                  "fxn_class"             "validated"            
[16] "docsum"                "tax_id"                "orig_build"           
[19] "upd_build"             "createdate"            "updatedate"           
[22] "ss"                    "allele"                "snp_class"            
[25] "chrpos"                "chrpos_prev_assm"      "text"                 
[28] "snp_id_sort"           "clinical_sort"         "cited_sort"           
[31] "chrpos_sort"           "merged_sort"          



In [51]:
### And let's filter for the actual values we want to extract: 
desired_dbSNP_values = (['genes', 'uid', 'snp_class', 
                         'fxn_class', 'chr', 'chrpos', 'global_mafs', 
                         'allele','clinical_significance', 'acc'] 
                         )
filtered_search_test_dict = {key: str(search_test.rx2(key)[0]) for key in search_test.names if key in desired_dbSNP_values}
filtered_search_test = pd.DataFrame([filtered_search_test_dict])
filtered_search_test

,uid,global_mafs,clinical_significance,genes,acc,chr,fxn_class,allele,snp_class,chrpos
0,4977574,"[1] ""1000Genomes"" ""ALSPAC"" ""Chilea...",risk-factor,"[1] ""CDKN2B-AS1""\n",NC_000009.12,9,"genic_downstream_transcript_variant,intron_var...",D,snv,9:22098575


In [52]:
filtered_and_ordered_search_test = filtered_search_test[[col for col in desired_dbSNP_values if col in filtered_search_test.columns]]
filtered_and_ordered_search_test


,genes,uid,snp_class,fxn_class,chr,chrpos,global_mafs,allele,clinical_significance,acc
0,"[1] ""CDKN2B-AS1""\n",4977574,snv,"genic_downstream_transcript_variant,intron_var...",9,9:22098575,"[1] ""1000Genomes"" ""ALSPAC"" ""Chilea...",D,risk-factor,NC_000009.12


In [ ]:
all_snp_data = []

# Open a text file in write mode to log skipped rows
with open("skipped_rows_log.txt", "w") as log_file:
    log_file.write("Details of rows skipped due to None UID:\n\n")

    for index, row in UID_SNP.iterrows():
        test_snp = row['UID']
        
        # Check if the UID is not None
        if test_snp is None:
            # Print to console and write to log file
            log_message = f"Skipping row {index} because UID is None. Row details:\n{row}\n\n"
            print(log_message)
            log_file.write(log_message)
            continue  # Skip this iteration if UID is None

        try: 
            # Retrieve SNP data from the SNP database
            search_test = rentrez.entrez_summary(db='snp', id=test_snp)
            
            # Filter and safely retrieve desired fields
            filtered_search_test_dict = {
                key: str(search_test.rx2(key)[0]) if len(search_test.rx2(key)) > 0 else None
                for key in search_test.names if key in desired_dbSNP_values
            }
            
            # Order the data to match the desired column order
            ordered_data = {key: filtered_search_test_dict.get(key, None) for key in desired_dbSNP_values}
            
            # Append ordered data to the results list
            all_snp_data.append(ordered_data)

        except Exception as e:
            error_message = f"Error retrieving data for UID {test_snp}: {e}\n"
            print(error_message)
            log_file.write(error_message)

# Convert results to DataFrame
final_df = pd.DataFrame(all_snp_data)

# Display the first 50 rows
final_df.head(50)


In [ ]:
#### Now let's make a copy of our RAW results:
final_df.to_csv(os.path.join(output_path, 'compiled_SNPs_RENTREZ_RAW.csv'), index=False)


In [ ]:
#### And let's make it beautiful by rearranging and sorting each column:
#### First, let's delete all variants that are not SNVs:
filtered_final_df = final_df.loc[final_df['snp_class'] == 'snv'] 
filtered_final_df

In [ ]:
### Now let's clean up la mugre of each cell in the genes column

# Define a function to clean up each cell
def clean_cell(value):
    if isinstance(value, str):
        # Remove `[1]`, quotation marks, and newline characters
        cleaned_value = re.sub(r'^\[\d+\]\s*"', '', value).replace('"', '').strip()
        return cleaned_value
    return value

# Apply the cleaning function to the 'genes' column using .loc
filtered_final_df.loc[:, 'genes'] = filtered_final_df['genes'].apply(clean_cell)

filtered_final_df.head(200)

In [ ]:
# Create a copy to avoid the SettingWithCopyWarning
filtered_final_df = filtered_final_df.copy()

# Rename columns
filtered_final_df.rename(columns={'genes': 'Gene ID (GENE_ID)', 'uid': 'dbSNP ID (RS)',
                                  'snp_class': 'SNP Class (SCLS)', 'fxn_class': 'Functional consequence class (FXN)', 
                                  'chr': 'Chromosome', 'chrpos': 'Chromosome base position (CPOS)', 
                                  'global_mafs': 'Minor Allele Frequency from global population (GMAF)',
                                  'clinical_significance': 'Associated clinical phenotypes (CLIN)',
                                  'acc': 'Nucleotide Accessions (ACCN)', 'allele': 'Allele'                                 
                                 }, inplace=True)

# IUPAC mapping dictionary for ambiguity codes
iupac_mapping = {
    "R": "A/G", "Y": "C/T", "S": "G/C", "W": "A/T",
    "K": "G/T", "M": "A/C", "B": "C/G/T", "D": "A/G/T",
    "H": "A/C/T", "V": "A/C/G", "N": "A/T/C/G"
}

# Define a function to translate IUPAC codes in the 'allele' column
def translate_allele(allele):
    if isinstance(allele, str):  # Check if the value is a string
        # Replace each IUPAC code with its corresponding alleles
        return "/".join([iupac_mapping.get(a, a) for a in allele])
    return allele

# Assume filtered_final_df is your DataFrame and 'allele' is the column with IUPAC codes
# Apply the translation function to the 'allele' column
filtered_final_df['Allele'] = filtered_final_df['Allele'].apply(translate_allele)

# Display the updated DataFrame
filtered_final_df.head(5)


In [35]:
import requests
import pandas as pd

# Function to get Gene ID and Gene Type from Ensembl REST API
def get_gene_info_via_ensembl(gene_symbol, species="homo_sapiens"):
    # Construct the URL for Ensembl's REST API
    url = f"https://rest.ensembl.org/lookup/symbol/{species}/{gene_symbol}"
    
    # Set the headers to request JSON format
    headers = {"Content-Type": "application/json"}
    
    try:
        # Send the GET request to Ensembl's API
        response = requests.get(url, headers=headers)
        
        # Check if the response is successful
        if response.status_code == 200:
            data = response.json()
            
            # Extract Gene ID and Gene Type
            gene_id = data.get("id", "Gene ID not available")
            gene_type = data.get("biotype", "Gene type not available")
            
            # Check if Gene Type is None and replace it
            if gene_type is None:
                gene_type = "No gene reported"
            
            print(f"Gene Symbol: {gene_symbol}, Ensembl Gene ID: {gene_id}, Gene Type: {gene_type}")
            return gene_id, gene_type
        elif response.status_code == 400:
            print(f"No gene found for symbol '{gene_symbol}' in Ensembl.")
            return "No gene found", "Intergenic (Review)"
        else:
            print(f"Failed to retrieve data for {gene_symbol}. HTTP status: {response.status_code}")
            return None, "Error retrieving data"
    except Exception as e:
        print(f"An error occurred for gene {gene_symbol}: {e}")
        return None, "Error retrieving data"

# Assume filtered_final_df is your existing DataFrame with the gene symbols in a column named 'Gene Symbol'
# Replace 'Gene Symbol' with the name of your actual column containing the gene symbols
filtered_final_df[['Ensembl Gene ID', 'Gene Type']] = filtered_final_df['Gene ID (GENE_ID)'].apply(
    lambda gene: pd.Series(get_gene_info_via_ensembl(gene))
)


Gene Symbol: CDKN2B-AS1, Ensembl Gene ID: ENSG00000240498, Gene Type: lncRNA
Gene Symbol: LPA, Ensembl Gene ID: ENSG00000198670, Gene Type: protein_coding
Gene Symbol: ZNF335, Ensembl Gene ID: ENSG00000198026, Gene Type: protein_coding
Gene Symbol: CELSR2, Ensembl Gene ID: ENSG00000143126, Gene Type: protein_coding
No gene found for symbol 'None' in Ensembl.
Gene Symbol: APOE, Ensembl Gene ID: ENSG00000130203, Gene Type: protein_coding
No gene found for symbol 'None' in Ensembl.
Gene Symbol: ADAMTS8, Ensembl Gene ID: ENSG00000134917, Gene Type: protein_coding
Gene Symbol: CHRNA5, Ensembl Gene ID: ENSG00000169684, Gene Type: protein_coding
Gene Symbol: CDKN2B-AS1, Ensembl Gene ID: ENSG00000240498, Gene Type: lncRNA
Gene Symbol: LPA, Ensembl Gene ID: ENSG00000198670, Gene Type: protein_coding
Gene Symbol: CDKN2B-AS1, Ensembl Gene ID: ENSG00000240498, Gene Type: lncRNA
No gene found for symbol 'None' in Ensembl.
Gene Symbol: ZPR1, Ensembl Gene ID: ENSG00000109917, Gene Type: protein_codin

In [40]:
####Now let's output this table.
filtered_final_df.to_csv(os.path.join(output_path, 'RENTREZ_NCBI_Compiled.csv'), index=False)


In [41]:
### Again unique genes
Unique_Genes = filtered_final_df['Gene ID (GENE_ID)'].value_counts().reset_index()
Unique_Genes.columns = ['Gene ID (GENE_ID)', 'Reported SNPs']
Unique_Genes.head(20)

,Gene ID (GENE_ID),Reported SNPs
0,CDKN2B-AS1,4
1,LPA,4
2,ADAMTS8,3
3,DAB2IP,3
4,IL6R,3
5,ERG,3
6,GDF7,2
7,LDLR LDLR-AS1,2
8,CDKN1A,2
9,PCSK9,2


In [42]:
# Assuming filtered_final_df is your DataFrame
# Filter rows where Gene Type is 'protein-coding'
protein_coding_df = filtered_final_df[filtered_final_df['Gene Type'] == 'protein_coding']

# Display the filtered DataFrame
protein_coding_df.head(10)


,Gene ID (GENE_ID),dbSNP ID (RS),SNP Class (SCLS),Functional consequence class (FXN),Chromosome,Chromosome base position (CPOS),Minor Allele Frequency from global population (GMAF),Allele,Associated clinical phenotypes (CLIN),Nucleotide Accessions (ACCN),Ensembl Gene ID,Gene Type
1,LPA,10455872,snv,intron_variant,6,6:160589086,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",A/G,benign,NC_000006.12,ENSG00000198670,protein_coding
2,ZNF335,3827066,snv,"downstream_transcript_variant,intron_variant,g...",20,20:45957384,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",C/T,benign,NC_000020.11,ENSG00000198026,protein_coding
3,CELSR2,12740374,snv,3_prime_UTR_variant,1,1:109274968,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",G/T,association,NC_000001.11,ENSG00000143126,protein_coding
5,APOE,429358,snv,"missense_variant,coding_sequence_variant",19,19:44908684,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",C/T,"pathogenic,risk-factor,association,conflicting...",NC_000019.10,ENSG00000130203,protein_coding
7,ADAMTS8,4936098,snv,intron_variant,11,11:130410772,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",A/G/T,,NC_000011.10,ENSG00000134917,protein_coding
8,CHRNA5,17486278,snv,intron_variant,15,15:78575140,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",A/C,,NC_000015.10,ENSG00000169684,protein_coding
10,LPA,140570886,snv,intron_variant,6,6:160591981,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",C/T,,NC_000006.12,ENSG00000198670,protein_coding
13,ZPR1,964184,snv,"3_prime_UTR_variant,genic_downstream_transcrip...",11,11:116778201,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",G/C,,NC_000011.10,ENSG00000109917,protein_coding
14,DAB2IP,7025486,snv,"intron_variant,genic_upstream_transcript_variant",9,9:121660124,"[1] ""1000Genomes"" ""ALSPAC"" ""Chilea...",A/G,,NC_000009.12,ENSG00000136848,protein_coding
15,PLCE1,731141,snv,intron_variant,10,10:94138924,"[1] ""1000Genomes"" ""ALSPAC"" ""Chilea...",A/G,,NC_000010.11,ENSG00000138193,protein_coding


In [43]:

# Assuming filtered_final_df is your DataFrame with gene symbols and gene types

# Filter rows with 'protein-coding' in the Gene Type column
protein_coding_df = filtered_final_df[filtered_final_df['Gene Type'] == 'protein_coding']

# Extract the 'Gene ID (GENE_ID)' column as a list
protein_coding_genes = protein_coding_df['Gene ID (GENE_ID)'].tolist()

# Define the output path for the text file
output_path_txt = f'{output_path}/protein_coding_genes.txt'  # You can set this to any desired path

# Save the list to a text file with each gene on a new line
with open(output_path_txt, "w") as file:
    for gene in protein_coding_genes:
        file.write(f"{gene}\n")

print(f"Protein-coding genes have been saved to '{output_path}'")

# Display the first few items in the list for verification
print(protein_coding_genes[:5])  # Shows the first 5 genes


Protein-coding genes have been saved to 'Resources/OutputFromExtractionCode'
['LPA', 'ZNF335', 'CELSR2', 'APOE', 'ADAMTS8']
